# 준비
라이브러리, 파라미터 세팅

In [13]:
import pandas as pd
import numpy as np
import os
import glob
import re

In [14]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/99_SubmissionFiles_for_Ensemble_NN/'

TEST_SIZE = 0.2
RANDOM_SEED = 42

# 데이터 불러오기

In [15]:
# 테스트(캐글) 데이터 로드
df_ensemble = pd.read_csv(DATA_IN_PATH + 'en_data.csv')
df_ensemble.drop('i_dialog', axis=1, inplace=True)
df_ensemble.drop('i_utterance', axis=1, inplace=True)
df_ensemble.drop('speaker', axis=1, inplace=True)
df_ensemble.columns = ['Id','utterance']  # 컬럼명 변경
df_ensemble.head()

,Id,utterance
0,0,"Alright, whadyou do with him?"
1,1,Oh! You're awake!
2,2,Then you gotta come clean with Ma! This is not...
3,3,"Yeah, but this is"
4,4,I don't wanna hear it! Now go to my room!


In [16]:
# 앙상블 대상 파일 확인
fnames = glob.glob(DATA_OUT_PATH+'*.csv')
fnames.sort()

for i in fnames:
    print(i.split('/')[-1])

FRIENDS_BERT_Tensorflow_MAXLN64_BATSZ64_EPOCH5.csv
FRIENDS_CNN.csv
FRIENDS_DNN.csv
FRIENDS_ELECTRA_large_MAXLN85_BATSZ32_EPOCH12-Copy1.csv
FRIENDS_LSTM.csv


In [17]:
len(fnames)

5

In [18]:
for i in range(len(fnames)):
    tmp = pd.read_csv(fnames[i])
    tmp.drop('Id', axis=1, inplace=True)
    df_ensemble = pd.concat([df_ensemble, tmp], axis=1)

In [19]:
df_ensemble

,Id,utterance,Predicted,Predicted,Predicted,Predicted,Predicted
0,0,"Alright, whadyou do with him?",neutral,neutral,neutral,neutral,neutral
1,1,Oh! You're awake!,joy,surprise,neutral,surprise,surprise
2,2,Then you gotta come clean with Ma! This is not...,non-neutral,neutral,neutral,anger,neutral
3,3,"Yeah, but this is",neutral,neutral,neutral,neutral,neutral
4,4,I don't wanna hear it! Now go to my room!,non-neutral,neutral,neutral,anger,neutral
...,...,...,...,...,...,...,...
1618,1618,Nooo.,neutral,neutral,non-neutral,non-neutral,neutral
1619,1619,"Hi, Kate!",joy,neutral,neutral,joy,neutral
1620,1620,"Hi, Lauren.",neutral,neutral,neutral,neutral,neutral
1621,1621,"Hi, Lauren.",neutral,neutral,neutral,neutral,neutral


# 모델 앙상블(Majority Voting) 방식

In [20]:
ensemble_result = []

for i in range(len(df_ensemble)):  # Id 별
    cnt_neutral = 0
    cnt_non_neutral = 0
    cnt_joy = 0
    cnt_sadness = 0
    cnt_fear = 0
    cnt_anger = 0
    cnt_surprise = 0
    cnt_disgust = 0
    
    for j in range(2,len(fnames)+2):  # 모델 별 (for j in range(2,len(fnames)+1): ?)
        if df_ensemble.iloc[i,j] == 'neutral':
            cnt_neutral += 1
        if df_ensemble.iloc[i,j] == 'non-neutral':
            cnt_non_neutral += 1
        if df_ensemble.iloc[i,j] == 'joy':
            cnt_joy += 1
        if df_ensemble.iloc[i,j] == 'sadness':
            cnt_sadness += 1
        if df_ensemble.iloc[i,j] == 'fear':
            cnt_fear += 1
        if df_ensemble.iloc[i,j] == 'anger':
            cnt_anger += 1
        if df_ensemble.iloc[i,j] == 'surprise':
            cnt_surprise += 1
        if df_ensemble.iloc[i,j] == 'disgust':
            cnt_disgust += 1            
    
    print(i, 'neutral:', cnt_neutral \
          ,  'non-neutral:', cnt_non_neutral \
          ,  'joy:', cnt_joy \
          ,  'sadness:', cnt_sadness \
          ,  'fear:', cnt_fear \
          ,  'anger:', cnt_anger \
          ,  'surprise:', cnt_surprise \
          ,  'disgust:', cnt_disgust \
         )

    # ? => 학습(기본으로 주어진) 데이터에서 빈도가 가장 높은 것에서 낮은 순으로 정렬
    if cnt_neutral >= 3:  ensemble_result.append('neutral')
    elif cnt_non_neutral >= 3:  ensemble_result.append('non-neutral')
    elif cnt_joy >= 3:  ensemble_result.append('joy')
    elif cnt_surprise >= 3:  ensemble_result.append('surprise')
    elif cnt_anger >= 3:  ensemble_result.append('anger')
    elif cnt_sadness >= 3:  ensemble_result.append('sadness')
    elif cnt_disgust >= 3:  ensemble_result.append('disgust')
    elif cnt_fear >= 3:  ensemble_result.append('fear')
    else:  ensemble_result.append('non-neutral')  # 특정 감정으로 단정할 수 없으며 중립으로 보기 애매하다고 세팅

df_ensemble["Ensemble_Result"] = ensemble_result

0 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1 neutral: 1 non-neutral: 0 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 3 disgust: 0
2 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 1 surprise: 0 disgust: 0
3 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
4 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 1 surprise: 0 disgust: 0
5 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 1 surprise: 0 disgust: 0
6 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
7 neutral: 3 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
8 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
9 neutral: 4 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
10 neutral: 3 non-neutral: 0 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
11 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 

105 neutral: 4 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
106 neutral: 3 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
107 neutral: 4 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
108 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
109 neutral: 1 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 1 surprise: 0 disgust: 0
110 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
111 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
112 neutral: 3 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
113 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
114 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
115 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
116 neutral: 1 non-neutral: 0 jo

202 neutral: 2 non-neutral: 0 joy: 3 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
203 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
204 neutral: 4 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
205 neutral: 1 non-neutral: 2 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
206 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
207 neutral: 2 non-neutral: 2 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
208 neutral: 4 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
209 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
210 neutral: 3 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
211 neutral: 2 non-neutral: 0 joy: 3 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
212 neutral: 3 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
213 neutral: 5 non-neutral: 0 jo

297 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
298 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
299 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
300 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
301 neutral: 2 non-neutral: 2 joy: 0 sadness: 1 fear: 0 anger: 0 surprise: 0 disgust: 0
302 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
303 neutral: 4 non-neutral: 0 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
304 neutral: 4 non-neutral: 0 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
305 neutral: 1 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
306 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
307 neutral: 0 non-neutral: 5 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
308 neutral: 3 non-neutral: 2 jo

396 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 1 surprise: 0 disgust: 0
397 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
398 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
399 neutral: 2 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
400 neutral: 2 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
401 neutral: 2 non-neutral: 1 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
402 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 1 surprise: 2 disgust: 0
403 neutral: 2 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 1 surprise: 0 disgust: 0
404 neutral: 2 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
405 neutral: 3 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
406 neutral: 2 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
407 neutral: 3 non-neutral: 1 jo

...

1228 neutral: 3 non-neutral: 0 joy: 1 sadness: 1 fear: 0 anger: 0 surprise: 0 disgust: 0
1229 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1230 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1231 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1232 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
1233 neutral: 2 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 2 surprise: 0 disgust: 0
1234 neutral: 3 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1235 neutral: 3 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1236 neutral: 3 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
1237 neutral: 3 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1238 neutral: 4 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1239 neutral: 3 non-n

1325 neutral: 2 non-neutral: 2 joy: 0 sadness: 1 fear: 0 anger: 0 surprise: 0 disgust: 0
1326 neutral: 1 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
1327 neutral: 3 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1328 neutral: 3 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
1329 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1330 neutral: 2 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1331 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1332 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1333 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1334 neutral: 3 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 1 surprise: 1 disgust: 0
1335 neutral: 2 non-neutral: 2 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1336 neutral: 3 non-n

1423 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1424 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1425 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
1426 neutral: 1 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1427 neutral: 4 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1428 neutral: 3 non-neutral: 1 joy: 0 sadness: 1 fear: 0 anger: 0 surprise: 0 disgust: 0
1429 neutral: 0 non-neutral: 0 joy: 4 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
1430 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1431 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
1432 neutral: 4 non-neutral: 0 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1433 neutral: 3 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1434 neutral: 3 non-n

1518 neutral: 3 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
1519 neutral: 4 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1520 neutral: 3 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
1521 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
1522 neutral: 1 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
1523 neutral: 2 non-neutral: 2 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1524 neutral: 4 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1525 neutral: 3 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1526 neutral: 3 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 2 surprise: 0 disgust: 0
1527 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1528 neutral: 3 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1529 neutral: 2 non-n

1615 neutral: 2 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1616 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
1617 neutral: 3 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1618 neutral: 3 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1619 neutral: 3 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1620 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1621 neutral: 5 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1622 neutral: 2 non-neutral: 0 joy: 3 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0


In [21]:
df_ensemble.head(20)

,Id,utterance,Predicted,Predicted,Predicted,Predicted,Predicted,Ensemble_Result
0,0,"Alright, whadyou do with him?",neutral,neutral,neutral,neutral,neutral,neutral
1,1,Oh! You're awake!,joy,surprise,neutral,surprise,surprise,surprise
2,2,Then you gotta come clean with Ma! This is not...,non-neutral,neutral,neutral,anger,neutral,neutral
3,3,"Yeah, but this is",neutral,neutral,neutral,neutral,neutral,neutral
4,4,I don't wanna hear it! Now go to my room!,non-neutral,neutral,neutral,anger,neutral,neutral
5,5,I don?t want him to tell this story for years.,non-neutral,neutral,neutral,anger,neutral,neutral
6,6,"Oh, but he will. He still tells the story how ...",neutral,neutral,neutral,neutral,neutral,neutral
7,7,I wasn?t escaping.,neutral,non-neutral,neutral,non-neutral,neutral,neutral
8,8,Then how did you get caught in the barbed wire?,surprise,neutral,neutral,non-neutral,neutral,neutral
9,9,I was trying to help out a squirrel.,non-neutral,neutral,neutral,neutral,neutral,neutral


In [22]:
df_ensemble.tail(20)

,Id,utterance,Predicted,Predicted,Predicted,Predicted,Predicted,Ensemble_Result
1603,1603,Yeah.,neutral,neutral,neutral,neutral,neutral,neutral
1604,1604,Hey!,joy,joy,neutral,joy,neutral,joy
1605,1605,Hey.,neutral,joy,neutral,neutral,neutral,neutral
1606,1606,What are you wrapping?,neutral,surprise,neutral,neutral,surprise,neutral
1607,1607,"Oh-ho, look what I got Julio.",neutral,surprise,non-neutral,non-neutral,surprise,non-neutral
1608,1608,"Yeah, it's a vase.",neutral,neutral,surprise,neutral,neutral,neutral
1609,1609,"Yeah, just like the one in the poem.",neutral,neutral,neutral,neutral,neutral,neutral
1610,1610,Well not exactly like the one in the poem.,neutral,neutral,neutral,neutral,neutral,neutral
1611,1611,What are you talking about?,surprise,surprise,neutral,surprise,surprise,surprise
1612,1612,It was a big deal.,neutral,neutral,neutral,anger,neutral,neutral


In [23]:
output = df_ensemble[['Id','Ensemble_Result']].copy()
output = output.rename({'Ensemble_Result':'Predicted'}, axis='columns')
output.head()

,Id,Predicted
0,0,neutral
1,1,surprise
2,2,neutral
3,3,neutral
4,4,neutral


In [24]:
# 해당 경로가 없으면 생성
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# csv파일 생성
output.to_csv("./data_out/FRIENDS_Ensemble_NN.csv", index = False)  # 앙상블 결과, 캐글 제출 용도

### 캐글 제출 결과
**[2020.12.20]**<br>
FRIENDS_BERT_Tensorflow_MAXLN64_BATSZ64_EPOCH5.csv<br>
FRIENDS_CNN.csv<br>
FRIENDS_DNN.csv<br>
FRIENDS_ELECTRA_large_MAXLN85_BATSZ32_EPOCH12.csv<br>
FRIENDS_LSTM.csv<br>
조합(4개) => 0.47595<br>

개별 모델 결과보다 성능 향상이 잘 되지 않음